In [1]:
from argparse import Namespace

In [2]:
import pytorch_lightning as pl

In [3]:
from src.data.make_harem import get_example_sets
from src.input.example import InputExample
from src.input.feature import convert_examples_to_features
from src.models.modeling_harem import T5ForHarem

In [ ]:
examples = get_example_sets('../data/harem/', merge_O=True)

In [4]:
hparams = {
    'merge_O': True,
    'labels_mode': 'words',
    'datapath': '../data/harem',
    'remove_accents': True,
    'max_length': 512,
    'stride': 256
}
hparams = Namespace(**hparams)

In [5]:
model = T5ForHarem.from_pretrained('t5-small', hparams=hparams)

In [18]:
examples = model.get_examples()

In [16]:
example = examples['test'][0]

In [17]:
example

Source: W. JAMES Willian James Willian James , filosofo e psicologo. Foi o mais influente dos pensadores dos EUA , criador do pragmatismo. Nasceu e, Nova Iorque , a 11 de Janeiro de 1842 . O seu pai, Henru James , era um teologo seguidor de Emanuel Swedenborg . Um dos seus irmaos foi o conhecido novelista Henry James . Concluiu os seus estudos de medicina, em 1870 , na Universidade de Harvard , onde iniciou a sua carreira como professor de fisiologia em 1872 . A partir de 1880 ensinou psicologia e filosofia em Harvard , universidade que abandonou em 1907 , proferindo conferencias nas universidades de Columbia e Oxford . Morreu em Chocorua , New Hampshire , a 26 de Agosto de 1910 . Obras Principios de Psicologia ( 1890 ), uma obra monumental que o projectou na comunidade cientifica e filosofica do tempo. A Vontade de Crer e Outros Ensaios Sobre Filosofia Popular ( 1897 ), A Imortalidade Humana ( 1898 ), Diversidade da Experiencia Religiosa ( 1902 ). Pragmatismo: um nome novo para velhas

In [10]:
max_target_length = 256
max_length = max_target_length - 1 # account for eos token

In [11]:
stride = 128

In [12]:
possible_endings = list(model.labels2words.values())

In [13]:
possible_endings

['[Outro]',
 '[Pessoa]',
 '[Local]',
 '[Tempo]',
 '[Valor]',
 '[Organização]',
 '[Ent]']

In [21]:
span_examples = []

for example in examples['train']:
    target = example.target
    target_tokens = model.tokenizer.tokenize(target)
    n_spans = len(target_tokens) // stride

    for i in range(n_spans):
        start = stride * i
        end = min(stride * i + max_length, len(target_tokens))

        span_tokens = target_tokens[start:end]
        target_span_words = model.tokenizer.convert_tokens_to_string(
            span_tokens).split(' ')
        last_ent_index = [i for i, w in enumerate(
            target_span_words) if w in possible_endings]
        
        if len(last_ent_index):
            last_ent_index = last_ent_index[-1]
            target_span_words = target_span_words[:last_ent_index+1]
        else:
            target_span_words = ['[Outro]']
            
        target_span_words = [
            w for w in target_span_words if w in example.target_words]
        source_span_words = [
            w for w in target_span_words if w in example.source_words]

        span_examples.append(InputExample(
            source_span_words, target_span_words))

In [22]:
target_span_words

['se',
 'assim',
 'o',
 'desejar',
 'profundamente.',
 '[Outro]',
 'Senhora',
 'Presidente',
 '[Pessoa]',
 ',',
 'verifico',
 'que',
 'a',
 'pergunta',
 'oral',
 'sobre',
 'professores',
 'de',
 'linguas,',
 'que',
 'foi',
 'inserida',
 'extremamente',
 'tarde',
 'na',
 'nossa',
 'ordem',
 'do',
 'dia',
 'de',
 'sexta',
 'feira,',
 'nao',
 'tem',
 'proposta',
 'de',
 'resolucao',
 'anexa.',
 'Admito',
 'que,',
 'dadas',
 'as',
 'circunstancias',
 'bastante',
 'caoticas',
 'desta',
 'segunda',
 'feira,',
 'apresentei',
 'demasiado',
 'tarde',
 'o',
 'pedido',
 'no',
 'sentido',
 'de',
 'se',
 'incluir',
 'uma',
 'proposta',
 'de',
 'resolucao.',
 'Trata',
 'se',
 'de',
 'um',
 'assunto',
 'importante,',
 'e',
 'gostaria',
 'de',
 'perguntar',
 'se,',
 'apos',
 'o',
 'encerramento',
 'do',
 'debate,',
 'se',
 'podera',
 'fixar',
 'um',
 'prazo',
 'limite',
 'para',
 'a',
 'apresentacao',
 'de',
 'propostas',
 'de',
 'resolucao',
 'sobre',
 'o',
 'tema',
 'em',
 'questao.',
 '[Outro]']

In [14]:
target = example.target
target_tokens = model.tokenizer.tokenize(target)

n_spans = len(target_tokens) // stride
n_spans

9

In [15]:
span_examples = []

for i in range(n_spans):
    start = stride * i
    end = min(stride * i + max_length, len(target_tokens))
    
    span_tokens = target_tokens[start:end]
    target_span_words = model.tokenizer.convert_tokens_to_string(span_tokens).split(' ')
    last_ent_index = [i for i, w in enumerate(target_span_words) if w in possible_endings][-1]
    target_span_words = target_span_words[:last_ent_index+1]
    target_span_words = [w for w in target_span_words if w in example.target_words]
    source_span_words = [w for w in target_span_words if w in example.source_words]
    
    span_examples.append(InputExample(source_span_words, target_span_words))

In [16]:
features = convert_examples_to_features(span_examples, model.tokenizer, max_length=256, prefix='Reconhecer Entidades:')

In [19]:
model.tokenizer.decode(features[1].source_token_ids)

'Reconhecer Entidades:. O seu pai, Henru James, era um teologo seguidor de Emanuel Swedenborg. Um dos seus irmaos foi o conhecido novelista Henry James. Concluiu os seus estudos de medicina, em 1870, na Universidade de Harvard, onde iniciou a sua carreira como professor de fisiologia em 1872. A partir de 1880 ensinou psicologia e filosofia em Harvard'

In [20]:
model.tokenizer.decode(features[1].target_ids)

'. O seu pai, [Outro] Henru James [Pessoa], era um teologo seguidor de [Outro] Emanuel Swedenborg [Pessoa]. Um dos seus irmaos foi o conhecido novelista [Outro] Henry James [Pessoa]. Concluiu os seus estudos de medicina, em [Outro] 1870 [Tempo], na [Outro] Universidade de Harvard [Organizaç ⁇ o], onde iniciou a sua carreira como professor de fisiologia em [Outro] 1872 [Tempo]. A partir de [Outro] 1880 [Tempo] ensinou psicologia e filosofia em [Outro] Harvard [Organizaç ⁇ o]'

# From model

In [6]:
model.prepare_data()

In [8]:
dl_test = model.test_dataloader()

In [9]:
batch = next(iter(dl_test))

In [10]:
target_ids = model.get_target_token_ids(batch)

In [15]:
model.tokenizer.decode(target_ids[0])

'W. JAMES [Pessoa] Willian James [Pessoa] Willian James [Pessoa], filosofo e psicologo. Foi o mais influente dos pensadores dos [Outro] EUA [Local], criador do pragmatismo. Nasceu e, [Outro] Nova Iorque [Local], a [Outro] 11 de Janeiro de 1842 [Tempo]. O seu pai, [Outro] Henru James [Pessoa], era um teologo seguidor de [Outro] Emanuel Swedenborg [Pessoa]. Um dos seus irmaos foi o conhecido novelista [Outro] Henry James [Pessoa]. Concluiu os seus estudos de medicina, em [Outro] 1870 [Tempo], na [Outro] Universidade de Harvard [Organizaç ⁇ o], onde iniciou a sua carreira como professor de fisiologia em [Outro] 1872 [Tempo]. A partir de [Outro] 1880 [Tempo] ensinou psicologia e filosofia em [Outro] Harvard [Organizaç ⁇ o], universidade que abandonou em [Outro] 1907 [Tempo], proferindo conferencias nas universidades de [Outro] Columbia [Organizaç ⁇ o] e [Outro] Oxford [Organizaç ⁇ o]. Morreu em [Outro] Chocorua [Local], [Outro] New Hampshire [Local], a [Outro] 26 de Agosto de 1910 [Tempo]'